In [4]:
## associate channels and filter relevant channels for SWR and spindles

# LOAD PACKAGES

from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path


#######################################################################
#######################################################################
#                                ### SWR ###                          #
#######################################################################
#######################################################################


# LOAD fILES CA1
folder_input = Path('Gaelle/Baseline_recording/GreenLinesOK/session2/OpenEphys/')
EMGbooleaninput = folder_input / f'EMGframeBoolean.pkl'
EMGboolean = pd.read_pickle(EMGbooleaninput)
LFPwakeremovedinput = folder_input / f'LFPwake0.npy'
LFPwakeremoved = np.load(LFPwakeremovedinput, mmap_mode= 'r')
# For long recordings, use LFPwakeremoved directly to nt bother with super long wake periods during assessment
Allinput = folder_input / f'LFPwake0.npy'
All = np.load(Allinput, mmap_mode= 'r')
CA1 = All[:, 31]#-All[:, 22]
CA1wakeremoved = LFPwakeremoved[:,31]#-LFPwakeremoved[:,22]

## BAND PASS FILTER - 120-200 Hz -

# Filtre parameter:
f_lowcut = 120.
f_hicut = 200.
fs = 1000
nyq = 0.5 * fs
N = 6                 # Filtre order
Wn = [f_lowcut/nyq,f_hicut/nyq]  # Nyquist frequency fraction

# Filtering:
b, a = signal.butter(N, Wn, 'band')
filt_CA1 = signal.filtfilt(b, a, CA1)
filt_CA1wakeremoved = signal.filtfilt(b, a, CA1wakeremoved)

# Continuous Wavelet Transfor and projection

# Parameter and computation of CWT
w = 10.
freq = np.linspace(120, 180, 80)
widths = w*fs / (2*freq*np.pi)
CA1NWcwt = signal.cwt(filt_CA1wakeremoved, signal.morlet2, widths, w=w)

# Projection calculation
absCA1NWcwt = np.absolute(CA1NWcwt)
proj_CA1NWcwt = np.mean(absCA1NWcwt, axis = 0)/60
sdproj_CA1cwt = np.std(proj_CA1NWcwt)
sd3proj_CA1cwt = sdproj_CA1cwt*6#3
#sd05proj_CA1cwt = sdproj_CA1cwt*0.5



## EXTRACTION SWRs

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths


## On signal without wake 
# 3 sd threshold
peaks, properties = find_peaks(proj_CA1NWcwt, prominence=1, width=20, height=sd3proj_CA1cwt)

# SWR boundaries taken at 70% from peak of intensity. This means that the SWRs with small amplitude will be longer than the big ones.
results_width = peak_widths(proj_CA1NWcwt, peaks, rel_height=0.7)

# Organise results in numpy array
peaks2 = peaks.reshape(len(peaks),1)
npresults_width = np.array(results_width).reshape(4,-1)
SWR_prop = np.append(peaks2, results_width).reshape(5,len(peaks2)).round()

projMaxP_cwtmg = np.max(CA1NWcwt, axis = 0)
projMaxF_cwtmg = np.argmax(CA1NWcwt, axis = 0) + 120


nb_SWR = np.arange(0,len(peaks),1)
data = np.zeros((len(peaks),4))

for tt in nb_SWR:
    SWR_start = int(SWR_prop[3,tt])
    SWR_stop = int(SWR_prop[4,tt])
    SWR_MaxP = projMaxP_cwtmg[SWR_start:SWR_stop]
    SWR_MaxF = projMaxF_cwtmg[SWR_start:SWR_stop]
    data[tt, 0] = max(SWR_MaxF).round()
    data[tt, 1] = max(SWR_MaxP).round()
    data[tt, 2] = round(sum(SWR_MaxF)/len(SWR_MaxF))
    data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))

param_SWR = pd.DataFrame(data, columns = ['Max freq', 'Max int', 'Avg freq', 'Avg int'])
tSWR_prop = SWR_prop.transpose()
pd_prop_SWR = pd.DataFrame(tSWR_prop, columns = ['peak time', 'Duration', 'peak amp', 'start time', 'end time'])
All_SWR = pd.concat([pd_prop_SWR, param_SWR], axis=1)

folder_output = Path('/mnt/data/ahay/')#Path('/crnldata/waking/audrey_hay/')
filename2 = folder_input / f'SWRproperties.pkl'
filename3 = folder_input / f'SWRproperties.csv'
All_SWR.to_pickle(filename2)
All_SWR.to_csv(filename3, sep = ',')

combined = np.stack([filt_CA1wakeremoved, proj_CA1NWcwt], axis = 1)
filenameC = folder_input / f'SignalCA1.npy'
np.save(filenameC, combined)

/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_70270/2974338850.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 1] = max(SWR_MaxP).round()
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_70270/2974338850.py:98: DeprecationWarning: The Python built-in `round` is deprecated for complex scalars, and will raise a `TypeError` in a future release. Use `np.round` or `scalar.round` instead.
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))
/var/folders/jq/nz6nvbd54ns2lkjvj31_7_4r0000gn/T/ipykernel_70270/2974338850.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  data[tt, 3] = round(sum(SWR_MaxP)/len(SWR_MaxP))
